In [1]:
from simulate_BC1 import *
from simulate_BCn import *
import subprocess as sp
from sample_select import *
import numpy as np
import pandas as pd
import json

In [2]:
node = {'State':[], 'Action':(), 'Reward':0.0}
script_prefix = '/home/lizy/crop/Simulate_Env/script/'
genes = pd.read_csv('./data/Corn_Genetic_Map.map', delimiter='\t')['marker'].to_numpy()
nums = np.arange(100, 301, 50)
percent = np.arange(20, 61)

In [18]:
def find_closest_values(arr, target):
    idx = np.searchsorted(arr, target)
    if idx == len(arr):
        lower = arr[-1]
        upper = np.inf
    elif idx == 0:
        lower = arr[0]
        upper = np.inf
    else:
        lower = arr[idx - 1]
        upper = arr[idx] if idx < len(arr) else np.inf
    return lower, upper

def p_compute(target_gene):
    chrom, pos_ = target_gene.split('_')
    pos_, left, right = int(pos_), int(int(pos_)-5.25e6), int(int(pos_)+5.25e6)
    gene_map = pd.read_csv('./data/10RIL_genetic_map_V3.txt', delimiter='\t')
    pro_map = pd.read_csv('./data/Corn_Genetic_Map.map', delimiter='\t').set_index('marker')
    target_p = pro_map.loc[target_gene, 'position']
    gene_map = gene_map.loc[gene_map['chr'] == int(chrom)]
    if left > 0:
        t1, t2 = find_closest_values(gene_map['pos_v3'].to_numpy(), left)
        if t2 == np.inf:
            pl = gene_map[gene_map['pos_v3'] == t1]['pos_g'].values[0]
            pl = left/t1 * pl
        else:
            l, r = gene_map[gene_map['pos_v3'] == t1]['pos_g'].values[0], gene_map[gene_map['pos_v3'] == t2]['pos_g'].values[0]
            pl = ((left-t1)/(t2-t1) * (r-l) + l)
    t1, t2 = find_closest_values(gene_map['pos_v3'].to_numpy(), right)
    if t2 == np.inf:
        pr = gene_map[gene_map['pos_v3'] == t1]['pos_g'].values[0]
        pr = right/t1 * pl
    else:
        l, r = gene_map[gene_map['pos_v3'] == t1]['pos_g'].values[0], gene_map[gene_map['pos_v3'] == t2]['pos_g'].values[0]
        pr = ((right-t1)/(t2-t1) * (r-l) + l)
    pl = (target_p - pl) / 100
    pr = (pr - target_p) / 100
    return pl, pr

In [4]:
def Generate_BC1(i):
    BC1_num = 1000
    simulate_BC1(BC1_num, script_prefix)
    sample_fliter(script_prefix,'BC1',target_gene)
    samples = pd.read_csv('./BCres/BC1F1/BC1F1.samples', delimiter='\t')
    samples = samples.sort_values(by='PRPG', ascending=False)
    samples.loc[samples['Length(left)'] < 5.25, 'pl'] = 1.0
    samples.loc[samples['Length(left)'] >= 5.25, 'pl'] = pl
    samples.loc[samples['Length(right)'] < 5.25, 'pr'] = 1.0
    samples.loc[samples['Length(right)'] >= 5.25, 'pr'] = pr
    act = [int(np.random.choice(percent)), int(np.random.choice(nums))]
    Nt = samples.loc[(samples['Length(left)'] < 5.25) & (samples['Length(right)'] < 5.25)].shape[0]
    node = {'State':[np.mean(samples['PRPG']), Nt, np.mean(samples['pl']), np.mean(samples['pr'])], 'Action': act, 'Reward': 0.0}
    seq[i].append(node.copy())
    return act, Nt, samples

def Generate_BCn(BCn, act, samples, i):
    P = samples.head(act[0])['samples'].to_list()
    simulate_BCn(P=P, BCn=BCn, BC_nums=[act[1]] * act[0], script_prefix=script_prefix)
    sample_fliter(script_prefix, BCn, target_gene)
    samples = pd.read_csv('./BCres/%sF1/%sF1.samples'%(BCn, BCn), delimiter='\t')
    samples = samples.sort_values(by='PRPG', ascending=False)
    samples.loc[samples['Length(left)'] < 5.25, 'pl'] = 1.0
    samples.loc[samples['Length(left)'] >= 5.25, 'pl'] = pl
    samples.loc[samples['Length(right)'] < 5.25, 'pr'] = 1.0
    samples.loc[samples['Length(right)'] >= 5.25, 'pr'] = pr
    act = [int(np.random.choice(percent)), int(np.random.choice(nums))]
    Nt = samples.loc[(samples['Length(left)'] < 5.25) & (samples['Length(right)'] < 5.25)].shape[0]
    node = {'State':[np.mean(samples['PRPG']), Nt, np.mean(samples['pl']), np.mean(samples['pr'])], 'Action': act, 'Reward': 0.0}
    seq[i].append(node.copy())
    return act, Nt, np.mean(samples['PRPG']), samples 

In [5]:
with open('./genes_5-50.json', 'r', encoding='utf-8') as f:
    genes = json.load(f)
pl, pr = 0.0, 0.0
target_gene = ''
print(len(genes))

45


In [20]:
for j in range(0, 5):
    target_gene = genes[j]
    pl, pr = p_compute(target_gene)
    seq = [[] for i in range(100)]
    print(j, target_gene, end=' \n|')
    for i in range(0, 100):
        act, Nt, samples = Generate_BC1(i)
        total_reward = (6*60*300) - 1000
        print('#', end='')
        for n in range(2, 7):
            if n > 3 and prpg > 0.985 and Nt >= 10:
                break
            elif n == 6 and (prpg < 0.985 or Nt <10):
                total_reward = 0.0
            act, Nt, prpg, samples = Generate_BCn('BC%d'%n, act, samples, i)
            total_reward -= act[0] * act[1]
            print('#', end='')

        for j in range(len(seq[i])):
            seq[i][j]['Reward'] = total_reward / 50
            
        print('_', end='')
    print('|\n', j, target_gene, 'already generate')
    with open('./sequence/%s.json'%target_gene, 'w', encoding='utf-8') as f:
        json.dump(seq, f, ensure_ascii=False)

0 8_88472086 
|####_####_####_######_####_####_####_####_####_####_#####_####_#####_####_####_####_####_#####_####_####_####_####_####_####_####_####_####_#####_####_####_####_####_####_####_####_####_####_####_#####_#####_####_#####_#####_####_####_####_####_####_####_####_####_####_####_####_####_#####_####_####_####_####_####_####_####_####_####_#####_####_####_####_#####_###_####_####_####_####_#####_####_####_#####_####_####_#####_#####_####_#####_####_#####_####_####_####_####_####_#####_####_####_####_#####_#####_####_####_|
 3 8_88472086 already generate
1 8_25355951 
|####_####_####_####_####_####_####_####_####_###_####_####_####_####_####_###_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_###_####_####_###_####_####_####_####_####_###_####_####_####_####_####_####_####_###_####_####_####_####_####_####_####_####_####_####_####_####_####_###_####_###_####_####_####_###_###_###_####_####_####_####_####_####_####_####_####_####_###_####_###

In [21]:
for j in range(5, 10):
    target_gene = genes[j]
    pl, pr = p_compute(target_gene)
    seq = [[] for i in range(100)]
    print(j, target_gene, end=' \n|')
    for i in range(0, 100):
        act, Nt, samples = Generate_BC1(i)
        total_reward = (6*60*300) - 1000
        print('#', end='')
        for n in range(2, 7):
            if n > 3 and prpg > 0.985 and Nt >= 10:
                break
            elif n == 6 and (prpg < 0.985 or Nt <10):
                total_reward = 0.0
            act, Nt, prpg, samples = Generate_BCn('BC%d'%n, act, samples, i)
            total_reward -= act[0] * act[1]
            print('#', end='')

        for j in range(len(seq[i])):
            seq[i][j]['Reward'] = total_reward / 50
            
        print('_', end='')
    print('|\n', j, target_gene, 'already generate')
    with open('./sequence/%s.json'%target_gene, 'w', encoding='utf-8') as f:
        json.dump(seq, f, ensure_ascii=False)

5 10_53548632 
|######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_######_|
 5 10_53548632 already generate
6 9_12797477 
|####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_####_

In [19]:
# with open('./genes_51-100.json', 'r', encoding='utf-8') as f:
#     genes = json.load(f)
# for j in range(50):
# target_gene = genes[j]
from sample_select import compute_pl_pr
target_gene = '1_135401720'
pl, pr = compute_pl_pr(target_gene)
pl, pr = p_compute(target_gene)
print(pl, pr)
seq = [[] for i in range(100)]
# print(i, target_gene, end=' \n|')
# for i in range(0, 1):
#     act, Nt, samples = Generate_BC1(i)
#     for n in range(2, 7):
#         if n > 4 and prpg > 0.985 and Nt >= 1:
#             break
#         act, Nt, prpg, samples = Generate_BCn('BC%d'%n, act, samples, i)
#         print('#', end='')
#     print('_', end='')
# print('|\n', i, target_gene, 'already generate')
# with open('./sequence/%s.json'%target_gene, 'w', encoding='utf-8') as f:
#     json.dump(seq, f, ensure_ascii=False)

0.0021252710714006182 0.003802291145436811


In [ ]:
target_gene = '1_135401720'
# target_gene = '3_230143954'
for i in range(0, 30):
    act, Nt, samples = Generate_BC1(i)
    total_reward = (6*60*300) - 1000
    print('#', end='')
    for n in range(2, 7):
        if n > 3 and prpg > 0.985 and Nt >= 10:
            break
        elif n == 6 and (prpg < 0.985 or Nt <10):
            total_reward = 0.0
        act, Nt, prpg, samples = Generate_BCn('BC%d'%n, act, samples, i)
        total_reward -= act[0] * act[1]
        print('#', end='')

    for j in range(len(seq[i])):
        seq[i][j]['Reward'] = total_reward / 50
        
    print('_', end='')
print('|\n', j, target_gene, 'already generate')
with open('./sequence_random/%s.json'%target_gene, 'w', encoding='utf-8') as f:
    json.dump(seq, f, ensure_ascii=False)

In [19]:
BC1_num = 1000
seq = [[] for i in range(100)]
target_gene = '8_88472086'
pl, pr = p_compute(target_gene)
simulate_BC1(BC1_num, script_prefix)
sample_fliter(script_prefix,'BC1',target_gene)
samples = pd.read_csv('./BCres/BC1F1/BC1F1.samples', delimiter='\t')
samples = samples.sort_values(by='PRPG', ascending=False)
for idx, row in samples.iterrows():
    if row['Length(left)'] < 5.25:
        samples.loc[idx, 'pl'] = 1.0
    else:
        samples.loc[idx, 'pl'] = pl / 100
    if row['Length(right)'] < 5.25:
        samples.loc[idx, 'pr'] = 1.0
    else:
        samples.loc[idx, 'pr'] = pr / 100
act = [int(np.random.choice(percent)), int(np.random.choice(nums))]
Nt = samples.loc[(samples['Length(left)'] < 5.25) & (samples['Length(right)'] < 5.25)].shape[0]
node = {'State':[np.mean(samples['PRPG']).round(4), Nt, np.mean(samples['pl']).round(4), np.mean(samples['pr']).round(4)], 'Action': act, 'Reward': 1/1000}
seq[0].append(node.copy())

In [28]:
seq

[[{'State': [0.7408, 0, 0.0, 0.0288], 'Action': [56, 100], 'Reward': 0.001},
  {'State': [0.9107, 2, 0.0064, 0.077],
   'Action': [46, 250],
   'Reward': 0.00015151515151515152},
  {'State': [0.9795, 2, 0.0042, 0.0438],
   'Action': [41, 250],
   'Reward': 5.524861878453039e-05},
  {'State': [0.9957, 141, 0.0812, 0.1925],
   'Action': [40, 300],
   'Reward': 3.527336860670194e-05}],
 [{'State': [0.7399, 0, 0.0039, 0.0347], 'Action': [54, 250], 'Reward': 0.001},
  {'State': [0.9102, 1, 0.0047, 0.0259],
   'Action': [49, 150],
   'Reward': 6.896551724137931e-05},
  {'State': [0.9813, 1, 0.006, 0.1069],
   'Action': [56, 300],
   'Reward': 4.576659038901602e-05}],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
